In [1]:
using Flux

In [3]:
mutable struct models
    model
    opt
    loss
end

In [4]:
struct netQAgt
    n_act::Int
    input_size::Int
    n_dense::Int
    ϵ::Float64
    γ::Float64
    filepath::String
end

function init_nQ()
    n_act::Int = 3
    input_size::Int = 4
    n_dense::Int = 32
    ϵ::Float64 = 0.1
    γ::Float64 = 0.9
    filepath::String = "test"

    return n_act, input_size, n_dense, ϵ, γ, filepath
end

init_nQ (generic function with 1 method)

In [13]:
function build_model(nq::netQAgt)
    model = Chain(Dense(nq.input_size, nq.n_dense, relu), Dense(nq.n_dense, nq.n_act))
    opt = ADAM()
    loss(x,y) = Flux.mse(model(x),y)

    return model, opt, loss
end


build_model (generic function with 1 method)

In [14]:
nq = netQAgt(init_nQ()...)

netQAgt(3, 4, 32, 0.1, 0.9, "test")

In [15]:
m_test = models(build_model(nq)...)

models(Chain(Dense(4 => 32, relu), Dense(32 => 3)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#3"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(4 => 32, relu), Dense(32 => 3))))

In [16]:
b = m_test.model(ones(4))

3-element Vector{Float64}:
  0.07427979263073353
  0.12010678845424419
 -0.4972820025765046

In [17]:
c = m_test.loss(ones(4),ones(3))

1.2910077870985366

In [18]:
println(m_test.model)

Chain(Dense(4 => 32, relu), Dense(32 => 3))


In [21]:
using BSON

In [23]:
bson("test.bson", m_test.model)

LoadError: MethodError: no method matching bson(::String, ::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}})
[0mClosest candidates are:
[0m  bson(::String; kws...) at ~/.julia/packages/BSON/rOaki/src/write.jl:89
[0m  bson(::String, [91m::AbstractDict[39m) at ~/.julia/packages/BSON/rOaki/src/write.jl:87

In [2]:
model = Chain(Dense(2, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

Chain(
  Dense(2 => 5, relu),                  # 15 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 27 parameters, 364 bytes.

In [3]:
function loss(x, y)
    return (x' * x - y^2)
end

loss (generic function with 1 method)

In [4]:
opt = ADAM()
data = [([1.0, 2.0], 0.5)]

1-element Vector{Tuple{Vector{Float64}, Float64}}:
 ([1.0, 2.0], 0.5)

In [5]:
Flux.train!(loss,Flux.params(model), data, opt)

In [26]:
using BSON: @save

In [37]:
typeof(model)

Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}}

In [27]:
@save "test.bson" model

In [28]:
m_test.model = Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

Chain(
  Dense(10 => 5, relu),                 # 55 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 67 parameters, 524 bytes.

In [38]:
m_test.model[1]

Dense(10 => 5, relu)  # 55 parameters

In [40]:
@save "test.bson" m_test.model

LoadError: LoadError: Unrecognised @save expression m_test.model
in expression starting at In[40]:1

In [44]:

bson("test2.bson", model)

LoadError: MethodError: no method matching bson(::String, ::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}})
[0mClosest candidates are:
[0m  bson(::String; kws...) at ~/.julia/packages/BSON/rOaki/src/write.jl:89
[0m  bson(::String, [91m::AbstractDict[39m) at ~/.julia/packages/BSON/rOaki/src/write.jl:87

In [49]:
@save "test2.bson" model

In [52]:
test = m_test.model |> cpu

Chain(
  Dense(10 => 5, relu),                 # 55 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 67 parameters, 524 bytes.

In [53]:
@save "test2.bson" test

In [54]:
using BSON: @load

In [59]:
m_test2 = models(build_model(nq)...)


models(Chain(Dense(4 => 32, relu), Dense(32 => 3)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#3"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(4 => 32, relu), Dense(32 => 3))))

In [69]:
@load "test2.bson" test

In [70]:
q = [1.0 2.0 3.0 5.0]

1×4 Matrix{Float64}:
 1.0  2.0  3.0  5.0

In [6]:
mutable struct models
    Kt
    model
    opt
    loss
end

In [24]:
function build_model()
    Kt = ones(Float64,2)
    model = Chain(Dense(2, 4, relu), Dense(4, 2))
    opt = ADAM()
    loss(v, x, y) = (model(x)' * model(x) - y^2 + sum(v))

    return Kt, model, opt, loss
end

build_model (generic function with 1 method)

In [25]:
m = models(build_model()...)

models([1.0, 1.0], Chain(Dense(2 => 4, relu), Dense(4 => 2)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#6"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(2 => 4, relu), Dense(4 => 2))))

In [29]:
Loss(x,y) = m.loss(m.Kt, x, y)
Flux.train!(Loss,Flux.params(m.model), data, m.opt)

In [15]:
Flux.params(m.model)

Params([Float32[0.010531068 -0.35521567; -0.91696393 -0.100117445; -0.2657094 -0.6087868; -0.7047603 0.23531306], Float32[0.0, 0.0, 0.0, 0.0], Float32[0.70487535 0.73120105 -0.19861972 0.3961799; -0.67083395 0.97618544 -0.87142813 0.42663658], Float32[0.0, 0.0]])

In [30]:
Loss([1.0,2.0], 0.5)

2.1899924207384402

In [31]:
m.Kt = [2.0, 1.0]

2-element Vector{Float64}:
 2.0
 1.0

In [32]:
Loss([1.0,2.0], 0.5)

3.1899924207384402